# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel. It makes use of the collection of plugins that have been registered to the kernel and using AI, will formulate a plan to execute a given ask.

Read more about it [here](https://aka.ms/sk/concepts/planner).

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.7.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview

### Setting Up Handlebars Planner
Handlebars Planner is located in the `Microsoft.SemanticKernel.Planning.Handlebars` package.

In [2]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var planner = new HandlebarsPlanner();

### Providing plugins to the planner
The planner needs to know what plugins are available to it. Here we'll import the `SummarizePlugin` and `WriterPlugin` we have defined on disk.

In [3]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "..", "samples", "plugins");

kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "SummarizePlugin"));
kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "WriterPlugin"));

Define your ASK. What do you want the Kernel to do?

In [4]:
#pragma warning disable SKEXP0060

var ask = "Next week is April 15th, the IRS day in the USA, I don't like that day so write something in the form of a poem.";
var originalPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Original plan:\n");
Console.WriteLine(originalPlan);

Original plan:

{{!-- Step 1: Create a string for the poem about IRS day --}}
{{set "poem" "On April 15th, IRS does lay, My heart sinks as tax is due today. Numbers and forms in a big array, Oh, how I wish you could just go away."}}

{{!-- Step 2: Print out the poem --}}
{{json poem}}


As you can see in the above plan, the Planner has taken the user's ask and converted it into a Plan object detailing how the AI would go about solving this task.

It makes use of the plugins that the Kernel has available to it and determines which functions to call in order to fulfill the user's ask.

Let's also define an inline plugin and have it be available to the Planner.
Be sure to give it a function name and plugin name.

In [5]:
string skPrompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
""";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

var shakespeareFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings, "Shakespeare");

Let's update our ask using this new plugin.

In [6]:
#pragma warning disable SKEXP0060

var ask = @"Next week is April 15th, the IRS day in the USA, I don't like that day so write something in the form of a poem using Shakespeare's style.";

var newPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Updated plan:\n");
Console.WriteLine(newPlan);

Updated plan:

{{!-- Step 1: Set the date for April 15th --}}
{{set "date" "April 15th"}}

{{!-- Step 2: Set the event description --}}
{{set "event" "IRS Day"}}

{{!-- Step 3: Set the poet's style --}}
{{set "style" "Shakespearean"}}

{{!-- Step 4: Generate a short poem in Shakespearean style --}}
{{set "poem" (WriterPlugin-ShortPoem input=(concat "On " date ", " event " approaches, a day of taxes and tears, " "to file or not to file, the question of fears. Alas, the IRS doth cometh, " "with wallets to drain, a day of dread, alas, the taxman to disdain."))}}

{{!-- Step 5: Return the generated poem --}}
{{json poem}}


### Executing the plans

Now that we have different plans, let's try to execute them! The Kernel can execute the plan using RunAsync.

In [7]:
#pragma warning disable SKEXP0060

var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("Original Plan results:\n");
Console.WriteLine(Utils.WordWrap(originalPlanResult.ToString(), 100));

Original Plan results:

On April 15th, IRS does lay, My heart sinks as tax is due today. Numbers and forms in a big array,
Oh, how I wish you could just go away.



Now lets execute and print the new plan:

In [8]:
#pragma warning disable SKEXP0060

var newPlanResult = await newPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("New Plan results:\n");
Console.WriteLine(newPlanResult);

New Plan results:

On IRS Day we all must beware,
For the taxman doth cometh with care.
To drain our wallets dry,
And make grown men cry,
Oh how we wish we could just disappear!

So gather your receipts and your forms,
And brace yourself for the storms.
For April 15th
